# 将轮对（车轴）的局部几何坐标，放置到轨道及全局坐标下去绘制

# 实现1

In [ ]:
%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = ['Noto Sans CJK JP']

# ========== 1) 轨道中心线数据 ==========

trajectory_data = np.load('trajectory_data.npz')
s_vals   = trajectory_data['s']      # 里程
xvals    = trajectory_data['x']      # 全局 X(s)
yvals    = trajectory_data['y']      # 全局 Y(s)
zvals    = trajectory_data['z']      # 全局 Z(s)
psi_vals = trajectory_data['psi']    # yaw
phi_vals = trajectory_data['phi']    # roll

left_rail = trajectory_data['left_rail']    # left_rail
right_rail = trajectory_data['right_rail']  # right_rail

def get_track_pose(s):
    idx = np.argmin(np.abs(s_vals - s))
    X_T = xvals[idx]
    Y_T = yvals[idx]
    Z_T = zvals[idx]
    yaw_T   = psi_vals[idx]
    pitch_T = 0.0     # 无坡度
    roll_T  = phi_vals[idx]
    return (X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T)

# ========== 2) 欧拉角 & 变换函数 ==========

def rot_z(yaw):
    c, s = np.cos(yaw), np.sin(yaw)
    return np.array([
        [ c, -s, 0],
        [ s,  c, 0],
        [ 0,  0, 1]
    ], dtype=float)

def rot_y(pitch):
    c, s = np.cos(pitch), np.sin(pitch)
    return np.array([
        [ c, 0,  s],
        [ 0, 1,  0],
        [-s, 0,  c]
    ], dtype=float)

def rot_x(roll):
    c, s = np.cos(roll), np.sin(roll)
    return np.array([
        [1,  0,  0],
        [0,  c, -s],
        [0,  s,  c]
    ], dtype=float)

def euler_zyx_to_matrix(yaw, pitch, roll):
    # Z-Y-X 顺序
    return rot_z(yaw) @ rot_y(pitch) @ rot_x(roll)

def make_transform(yaw, pitch, roll, px, py, pz):
    T = np.eye(4)
    R = euler_zyx_to_matrix(yaw, pitch, roll)
    T[:3,:3] = R
    T[:3, 3] = [px, py, pz]
    return T

def transform_points_3d(T, xyz):
    homo = np.hstack([xyz, np.ones((len(xyz),1))])  # (N,4)
    trans = (T @ homo.T).T                         # (N,4)
    return trans[:,:3]

# ========== 3) 轮对圆柱网格 (W系轴向沿 Y) ==========

def create_cylinder_mesh(length=2.0, radius=0.065, ntheta=20, nL=10):
    y_ = np.linspace(-length/2, length/2, nL)
    th = np.linspace(0, 2*np.pi, ntheta)
    TH, Y = np.meshgrid(th, y_)
    X = radius * np.cos(TH)
    Z = radius * np.sin(TH)
    return X, Y, Z

Xc, Yc, Zc = create_cylinder_mesh(length=2.0, radius=0.065)
pts_cyl = np.column_stack([Xc.ravel(), Yc.ravel(), Zc.ravel()])  # (N,3)

# ========== 4) 读取 SIMPACK 输出并将 y_ws01_x 重命名为 s_val ==========

def read_wheelset_6dof_with_s(file_path):
    chunks = pd.read_csv(file_path, sep='\t', float_precision='high', chunksize=10000)
    df = pd.DataFrame()
    for c in chunks:
        df = pd.concat([df, c], ignore_index=True)
    if 'y_ws01_x' in df.columns:
        df.rename(columns={'y_ws01_x': 's_val'}, inplace=True)
    return df

file_path = '/home/yaoyao/Documents/myProjects/ROS2WithSPCK/PostAnalysis/Result_Y_RosRt.log'
df_wheel = read_wheelset_6dof_with_s(file_path)

# ========== 5) 主绘图：两个子图 ==========

fig = plt.figure(figsize=(14,6))

# (A) ax1：宏观视图
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot(xvals, yvals, zvals, color='black', label='Track Centerline (Global)')
ax1.plot(left_rail[:,0], left_rail[:,1], left_rail[:,2], 'r-', label='Left Rail')
ax1.plot(right_rail[:,0], right_rail[:,1], right_rail[:,2], 'b-', label='Right Rail')

# 选取10帧多次绘制
N = len(df_wheel)
num_frames = 10
idxs = np.linspace(0, N-1, num_frames, dtype=int)

# 为了在 ax2 画“特写”，我们先假设要取最后一帧 idxs[-1] 做放大，也可以改成 idxs[0] 或中间帧
focus_idx = idxs[-1]  # 就以最后一帧举例

wheel_centers = []  # 用于存储每一帧车轴中心点(全局坐标)

for i, idx in enumerate(idxs):
    row = df_wheel.iloc[idx]
    s_  = row['s_val']
    yW  = row['y_ws01_y']
    zW  = row['y_ws01_z']
    rW  = row['y_ws01_roll']
    pW  = row['y_ws01_pitch']
    ywW = row['y_ws01_yaw']

    # 轨道->全局
    X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(s_)
    T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)

    # 轮对->轨道
    T_W2T = make_transform(ywW, pW, rW, 0.0, yW, zW)

    # 轮对->全局
    T_W2G = T_T2G @ T_W2T

    # 画多帧车轴在 ax1
    pts_world = transform_points_3d(T_W2G, pts_cyl)
    Xw = pts_world[:,0].reshape(Xc.shape)
    Yw = pts_world[:,1].reshape(Xc.shape)
    Zw = pts_world[:,2].reshape(Xc.shape)

    ax1.plot_surface(
        Xw, Yw, Zw,
        color='magenta',
        alpha=0.8,
        edgecolor='none'
    )

    # 记录这帧“车轴中心” (W系原点) 在全局下的坐标
    #   即对 (0,0,0) 做 T_W2G 变换
    #   也可直接看 T_W2G[:3,3]
    center = T_W2G[:3, 3]
    wheel_centers.append(center)

ax1.set_xlabel("X [m]")
ax1.set_ylabel("Y [m]")
ax1.set_zlabel("Z [m]")
ax1.set_title("Global View (Track + Wheelsets)")
ax1.legend()

# 让 ax1 也做等比例 (如你所需)
x_min, x_max = xvals.min(), xvals.max()
y_min, y_max = yvals.min(), yvals.max()
z_min, z_max = zvals.min(), zvals.max()
cx = 0.5*(x_min + x_max)
cy = 0.5*(y_min + y_max)
cz = 0.5*(z_min + z_max)
max_range = max(x_max - x_min, y_max - y_min, z_max - z_min) / 2
ax1.set_xlim(cx - max_range, cx + max_range)
ax1.set_ylim(cy - max_range, cy + max_range)
ax1.set_zlim(cz - max_range, cz + max_range)
ax1.set_box_aspect((1,1,1))
ax1.view_init(elev=30, azim=-60)


# (B) ax2：单个轮对特写
ax2 = fig.add_subplot(122, projection='3d')
# 同样先画轨道(可选)
ax2.plot(xvals, yvals, zvals, color='black', alpha=0.3, label='Track Centerline (Global)')
ax2.plot(left_rail[:,0], left_rail[:,1], left_rail[:,2], 'r-', label='Left Rail')
ax2.plot(right_rail[:,0], right_rail[:,1], right_rail[:,2], 'b-', label='Right Rail')

# 选定 focus_idx 帧, 即“特写对象”
rowF = df_wheel.iloc[focus_idx]
sF   = rowF['s_val']
yF   = rowF['y_ws01_y']
zF   = rowF['y_ws01_z']
rF   = rowF['y_ws01_roll']
pF   = rowF['y_ws01_pitch']
ywF  = rowF['y_ws01_yaw']

# 轨道->全局
X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(sF)
T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)

# 轮对->轨道
T_W2T = make_transform(ywF, pF, rF, 0.0, yF, zF)
# 轮对->全局
T_W2G = T_T2G @ T_W2T

pts_worldF = transform_points_3d(T_W2G, pts_cyl)
Xwf = pts_worldF[:,0].reshape(Xc.shape)
Ywf = pts_worldF[:,1].reshape(Xc.shape)
Zwf = pts_worldF[:,2].reshape(Xc.shape)

ax2.plot_surface(
    Xwf, Ywf, Zwf,
    color='magenta',
    alpha=0.9,
    edgecolor='none',
    label='Focused Wheelset'
)

# (B.1) 让 ax2 只显示“附近一小块范围”
#   取车轴中心:
center_global = T_W2G[:3, 3]  # shape=(3,)
margin = 1.5  # 只在车轴附近留 1.5m 的立方范围
ax2.set_xlim(center_global[0] - margin, center_global[0] + margin)
ax2.set_ylim(center_global[1] - margin, center_global[1] + margin)
ax2.set_zlim(center_global[2] - margin, center_global[2] + margin)

ax2.set_xlabel("X [m]")
ax2.set_ylabel("Y [m]")
ax2.set_zlabel("Z [m]")
ax2.set_title(f"单个轮对特写 (Frame={focus_idx})")
ax2.legend()

# (B.2) 等比例 & 视角
ax2.set_box_aspect((1,1,1))
ax2.view_init(elev=20, azim=30)

plt.tight_layout()
plt.show()

# 实现2

In [ ]:
%matplotlib widget

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = ['Noto Sans CJK JP']

# ========== 1) 轨道中心线数据 ==========

trajectory_data = np.load('trajectory_data.npz')
s_vals   = trajectory_data['s']      # 里程
xvals    = trajectory_data['x']      # 全局 X(s)
yvals    = trajectory_data['y']      # 全局 Y(s)
zvals    = trajectory_data['z']      # 全局 Z(s)
psi_vals = trajectory_data['psi']    # yaw
phi_vals = trajectory_data['phi']    # roll

left_rail = trajectory_data['left_rail']    # left_rail
right_rail = trajectory_data['right_rail']  # right_rail

def get_track_pose(s):
    idx = np.argmin(np.abs(s_vals - s))
    X_T = xvals[idx]
    Y_T = yvals[idx]
    Z_T = zvals[idx]
    yaw_T   = psi_vals[idx]
    pitch_T = 0.0     # 无坡度
    roll_T  = phi_vals[idx]
    return (X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T)

# ========== 2) 欧拉角 & 变换函数 ==========

def rot_z(yaw):
    c, s = np.cos(yaw), np.sin(yaw)
    return np.array([
        [ c, -s, 0],
        [ s,  c, 0],
        [ 0,  0, 1]
    ], dtype=float)

def rot_y(pitch):
    c, s = np.cos(pitch), np.sin(pitch)
    return np.array([
        [ c, 0,  s],
        [ 0, 1,  0],
        [-s, 0,  c]
    ], dtype=float)

def rot_x(roll):
    c, s = np.cos(roll), np.sin(roll)
    return np.array([
        [1,  0,  0],
        [0,  c, -s],
        [0,  s,  c]
    ], dtype=float)

def euler_zyx_to_matrix(yaw, pitch, roll):
    # Z-Y-X 顺序
    return rot_z(yaw) @ rot_y(pitch) @ rot_x(roll)

def make_transform(yaw, pitch, roll, px, py, pz):
    T = np.eye(4)
    R = euler_zyx_to_matrix(yaw, pitch, roll)
    T[:3,:3] = R
    T[:3, 3] = [px, py, pz]
    return T

def transform_points_3d(T, xyz):
    homo = np.hstack([xyz, np.ones((len(xyz),1))])  # (N,4)
    trans = (T @ homo.T).T                         # (N,4)
    return trans[:,:3]

# ========== 3) 轮对圆柱网格 (W系轴向沿 Y) ==========

def create_cylinder_mesh(length, radius, ntheta=30, nL=10):
    """
    生成沿Y轴的圆柱, [y in -length/2..+length/2], 半径=radius
    """
    y_ = np.linspace(-length/2, length/2, nL)
    th = np.linspace(0, 2*np.pi, ntheta)
    TH, Y = np.meshgrid(th, y_)
    X = radius * np.cos(TH)
    Z = radius * np.sin(TH)
    return X, Y, Z

# 1) 车轴 (Axle)
X_axle, Y_axle, Z_axle = create_cylinder_mesh(length=2.0, radius=0.065)
pts_axle = np.column_stack([X_axle.ravel(), Y_axle.ravel(), Z_axle.ravel()])

# 2) 车轮 (Wheel), 直径0.86 => radius=0.43, 厚度0.04 => length=0.04
wheel_radius = 0.43
wheel_thickness = 0.04
X_wh, Y_wh, Z_wh = create_cylinder_mesh(length=wheel_thickness, radius=wheel_radius)
pts_wheel = np.column_stack([X_wh.ravel(), Y_wh.ravel(), Z_wh.ravel()])

# 轮对中心中, 两个车轮中心分别位于 y= ±(1.435/2)= ±0.7175
wheel_offset = 0.7175

# ========== 4) 读取 SIMPACK 输出并将 y_ws01_x 重命名为 s_val ==========

def read_wheelset_6dof_with_s(file_path):
    chunks = pd.read_csv(file_path, sep='\t', float_precision='high', chunksize=10000)
    df = pd.DataFrame()
    for c in chunks:
        df = pd.concat([df, c], ignore_index=True)
    if 'y_ws01_x' in df.columns:
        df.rename(columns={'y_ws01_x': 's_val'}, inplace=True)
    return df

file_path = '/home/yaoyao/Documents/myProjects/ROS2WithSPCK/PostAnalysis/Result_Y_RosRt.log'
df_wheel = read_wheelset_6dof_with_s(file_path)

# ========== E) 开始绘图 ==========

fig = plt.figure(figsize=(14,6))

# (E.1) ax1: 大范围全局视图
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot(xvals, yvals, zvals, color='black', alpha=0.1, label='Track Centerline')
ax1.plot(left_rail[:,0], left_rail[:,1], left_rail[:,2], 'r-', alpha=0.1, label='Left Rail')
ax1.plot(right_rail[:,0], right_rail[:,1], right_rail[:,2], 'b-', alpha=0.1, label='Right Rail')

# 抽取10个时刻
N = len(df_wheel)
num_frames = 20
idxs = np.linspace(0, N-1, num_frames, dtype=int)

for i, idx in enumerate(idxs):
    row = df_wheel.iloc[idx]
    s_  = row['s_val']
    yW  = row['y_ws01_y']
    zW  = row['y_ws01_z']
    rW  = row['y_ws01_roll']
    pW  = row['y_ws01_pitch']
    ywW = row['y_ws01_yaw']

    # (1) 轨道->全局
    X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(s_)
    T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)

    # (2) 轮对->轨道
    T_W2T = make_transform(ywW, pW, rW, 0.0, yW, zW)

    # (3) 轮对->全局
    T_W2G = T_T2G @ T_W2T

    # ========== 在轮对局部系(W)下，分别画 1 根车轴+2 个车轮 ==========

    # 3a) 车轴
    ptsA = transform_points_3d(T_W2G, pts_axle)
    
    # 3b) 左轮 (中心y= +wheel_offset)
    pts_left_wheel_local = pts_wheel.copy()
    # 在局部系W中，把其整体平移到 y=+0.7175
    pts_left_wheel_local[:,1] += +wheel_offset
    ptsWL = transform_points_3d(T_W2G, pts_left_wheel_local)

    # 3c) 右轮 (中心y= -wheel_offset)
    pts_right_wheel_local = pts_wheel.copy()
    pts_right_wheel_local[:,1] += -wheel_offset
    ptsWR = transform_points_3d(T_W2G, pts_right_wheel_local)

    # (5) 绘图: 先画车轴, 再左轮, 再右轮
    Xaw = ptsA[:,0].reshape(X_axle.shape)
    Yaw = ptsA[:,1].reshape(X_axle.shape)
    Zaw = ptsA[:,2].reshape(X_axle.shape)
    ax1.plot_surface(Xaw, Yaw, Zaw, color='magenta', alpha=0.8, edgecolor='none')

    Xlw = ptsWL[:,0].reshape(X_wh.shape)
    Ylw = ptsWL[:,1].reshape(X_wh.shape)
    Zlw = ptsWL[:,2].reshape(X_wh.shape)
    ax1.plot_surface(Xlw, Ylw, Zlw, color='cyan', alpha=0.8, edgecolor='none')

    Xrw = ptsWR[:,0].reshape(X_wh.shape)
    Yrw = ptsWR[:,1].reshape(X_wh.shape)
    Zrw = ptsWR[:,2].reshape(X_wh.shape)
    ax1.plot_surface(Xrw, Yrw, Zrw, color='cyan', alpha=0.8, edgecolor='none')

ax1.set_xlabel("X [m]")
ax1.set_ylabel("Y [m]")
ax1.set_zlabel("Z [m]")
ax1.set_title("Global View (z Down, with Wheels)")

ax1.legend()

# 若想强制等比例，可设置:
x_min, x_max = xvals.min(), xvals.max()
y_min, y_max = yvals.min(), yvals.max()
z_min, z_max = zvals.min(), zvals.max()
cx = 0.5*(x_min + x_max)
cy = 0.5*(y_min + y_max)
cz = 0.5*(z_min + z_max)
max_range = max(x_max - x_min, y_max - y_min, z_max - z_min)*0.5
ax1.set_xlim(cx - max_range, cx + max_range)
ax1.set_ylim(cy - max_range, cy + max_range)
ax1.set_zlim(cz - max_range, cz + max_range)
ax1.set_box_aspect((1,1,1))
# ax1 (全局视图) z轴反转
ax1.invert_zaxis()  # 直接反转z轴方向
ax1.view_init(elev=30, azim=-60)

# ========== (E.2) ax2: 选取最后一帧做“特写视图” ==========

ax2 = fig.add_subplot(122, projection='3d')
ax2.plot(xvals, yvals, zvals, color='black', alpha=0.8, label='Track Centerline')
ax2.plot(left_rail[:,0], left_rail[:,1], left_rail[:,2], 'r-', alpha=0.8, label='Left Rail')
ax2.plot(right_rail[:,0], right_rail[:,1], right_rail[:,2], 'b-', alpha=0.8, label='Right Rail')

# 假设 focus_idx = idxs[-1]
focus_idx = idxs[-1]

rowF = df_wheel.iloc[focus_idx]
sF  = rowF['s_val']
yF  = rowF['y_ws01_y']
zF  = rowF['y_ws01_z']
rF  = rowF['y_ws01_roll']
pF  = rowF['y_ws01_pitch']
ywF = rowF['y_ws01_yaw']

# 轨道->全局
X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(sF)
T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)

# 轮对->轨道
T_W2T = make_transform(ywF, pF, rF, 0.0, yF, zF)

# 轮对->全局
T_W2G = T_T2G @ T_W2T

# 画 车轴 + 轮子
ptsA_F = transform_points_3d(T_W2G, pts_axle)

pts_wheel_LF = pts_wheel.copy()
pts_wheel_LF[:,1] += +wheel_offset
ptsWL_F = transform_points_3d(T_W2G, pts_wheel_LF)

pts_wheel_RF = pts_wheel.copy()
pts_wheel_RF[:,1] += -wheel_offset
ptsWR_F = transform_points_3d(T_W2G, pts_wheel_RF)

# 同样可以将他们合并, 算 bounding box:
pts_allF = np.concatenate([ptsA_F, ptsWL_F, ptsWR_F], axis=0)

Xmin, Xmax = pts_allF[:,0].min(), pts_allF[:,0].max()
Ymin, Ymax = pts_allF[:,1].min(), pts_allF[:,1].max()
Zmin, Zmax = pts_allF[:,2].min(), pts_allF[:,2].max()

Xmid = 0.5*(Xmin + Xmax)
Ymid = 0.5*(Ymin + Ymax)
Zmid = 0.5*(Zmin + Zmax)

box_half_size = 0.5 * max(Xmax - Xmin, Ymax - Ymin, Zmax - Zmin)
box_margin = 0.2
box_half_size += box_margin

ax2.set_xlim(Xmid - box_half_size, Xmid + box_half_size)
ax2.set_ylim(Ymid - box_half_size, Ymid + box_half_size)
ax2.set_zlim(Zmid - box_half_size, Zmid + box_half_size)

# 再实际画 surface:
XaF = ptsA_F[:,0].reshape(X_axle.shape)
YaF = ptsA_F[:,1].reshape(X_axle.shape)
ZaF = ptsA_F[:,2].reshape(X_axle.shape)
ax2.plot_surface(XaF, YaF, ZaF, color='magenta', alpha=0.8, edgecolor='none')

XlF = ptsWL_F[:,0].reshape(X_wh.shape)
YlF = ptsWL_F[:,1].reshape(X_wh.shape)
ZlF = ptsWL_F[:,2].reshape(X_wh.shape)
ax2.plot_surface(XlF, YlF, ZlF, color='cyan', alpha=0.8, edgecolor='none')

XrF = ptsWR_F[:,0].reshape(X_wh.shape)
YrF = ptsWR_F[:,1].reshape(X_wh.shape)
ZrF = ptsWR_F[:,2].reshape(X_wh.shape)
ax2.plot_surface(XrF, YrF, ZrF, color='cyan', alpha=0.8, edgecolor='none')

ax2.set_xlabel("X [m]")
ax2.set_ylabel("Y [m]")
ax2.set_zlabel("Z [m]")
# ax2 (特写视图) z轴反转
ax2.invert_zaxis()  # 直接反转z轴方向

ax2.set_title(f"单个轮对特写 (Frame={focus_idx})")

ax2.set_box_aspect((1,1,1))
ax2.view_init(elev=90, azim=-90, roll=-180)

plt.tight_layout()
plt.show()